In [9]:
import numpy as np
import librosa
import os
from pathlib import Path
import encoder.inference as encoder
import encoder.audio as audio
import librosa
from IPython.display import Audio

In [10]:
modelPath = Path('pretrained.pt')
encoder.load_model(modelPath)

Loaded encoder "pretrained.pt" trained to step 1564501
